In [32]:
import numpy as np
import pandas as pd
import os
import shutil
import sklearn.decomposition
# import qtl_config_utils
import sys

In [33]:
outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/timepoint_D52/input_files_untreated/pseudobulk/"

In [34]:
# general set up
genotypes_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered'
kinship_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered.rel'
annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/ensembl_gene_id_annos.tsv'
#annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/hgnc_symbol_annos.tsv'
chunk_file = '/nfs/leia/research/stegle/mjbonder/ChunkFiles/Ensembl_75_Limix_Annotation_FC_Gene_step100.txt'
n_pcs = 15
base_dir = '/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/timepoint_D52/'

In [35]:
sample_mapping_file = os.path.join(outdir, 'samplemapping.tsv')
phenotype_file = os.path.join(outdir, 'phenotypes.tsv')
covariates_file = os.path.join(outdir, 'covariates.tsv')
noise_matrix_file = os.path.join(outdir, 'noise_matrix.tsv')
# config_file = os.path.join(outdir, 'qtl_config.yaml')

In [36]:
# eQTL discovery parameters
#number of top expressed genes to test
n_genes = 50000

In [37]:
if not os.path.exists(outdir):
    os.makedirs(outdir, exist_ok=True)

In [38]:
mydir = '/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/'
D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.all_cells.untreated.groupedby.donor_id.tsv'

In [39]:
phenotype_input_file = mydir + D52

In [40]:
# phenotype_input_file = sys.argv[1]
# outdir = sys.argv[2]
# if len(sys.argv)>=4:
#     subset_field = sys.argv[3]
#     selected_subset = sys.argv[4]
# else:
#     subset_field = None
#     selected_subset = None

In [41]:
#load map HGNC to ensembl
mapping_df = pd.read_csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')

In [42]:
# process phenotype dataframe (untreated)
groupedby_df = pd.read_csv(phenotype_input_file, sep='\t')

In [43]:
groupedby_df.head()

,donor_id,n_cells,ATG10,TGDS,MIB2,NRBP1,MFI2-AS1,RAB11B,CHMP5,VAT1,...,GRIA2,ZNF268,AC003102.3,ATG3,NLE1,RP11-332H14.2,PJA1,BNIP2,ALKBH4,RUFY2
0,HPSI0813i-mioj_1,2004,0.112087,0.101908,0.034625,0.188004,0.063741,0.619833,1.093340,0.364345,...,0.173778,0.060328,0.022613,0.329515,0.028899,0.043251,0.237618,0.232301,0.087212,0.293700
1,HPSI1013i-zagm_1,22,0.244225,0.058878,0.364975,0.291192,0.025435,0.798470,0.803363,0.342263,...,0.120239,0.000000,0.000000,0.469690,0.029293,0.029017,0.114851,0.286578,0.188539,0.231858
2,HPSI0614i-lirf_5,451,0.103079,0.116142,0.220130,0.478572,0.067239,0.894777,0.979164,0.387981,...,0.153317,0.099130,0.017873,0.342599,0.036244,0.063664,0.254674,0.192429,0.162536,0.324287
3,HPSI0813i-wots_3,380,0.170352,0.057083,0.207481,0.515186,0.075978,0.994246,1.021121,0.591241,...,0.315672,0.077485,0.025384,0.404380,0.036955,0.037978,0.378846,0.137812,0.125242,0.275611
4,HPSI0513i-dipe_1,475,0.206783,0.063727,0.195588,0.431934,0.088978,0.974636,0.988321,0.447228,...,0.283717,0.055024,0.026170,0.328487,0.034488,0.054598,0.245008,0.191966,0.126739,0.239621


In [44]:
phenotype_df = groupedby_df.copy()

In [45]:
phenotype_df.head()

,donor_id,n_cells,ATG10,TGDS,MIB2,NRBP1,MFI2-AS1,RAB11B,CHMP5,VAT1,...,GRIA2,ZNF268,AC003102.3,ATG3,NLE1,RP11-332H14.2,PJA1,BNIP2,ALKBH4,RUFY2
0,HPSI0813i-mioj_1,2004,0.112087,0.101908,0.034625,0.188004,0.063741,0.619833,1.093340,0.364345,...,0.173778,0.060328,0.022613,0.329515,0.028899,0.043251,0.237618,0.232301,0.087212,0.293700
1,HPSI1013i-zagm_1,22,0.244225,0.058878,0.364975,0.291192,0.025435,0.798470,0.803363,0.342263,...,0.120239,0.000000,0.000000,0.469690,0.029293,0.029017,0.114851,0.286578,0.188539,0.231858
2,HPSI0614i-lirf_5,451,0.103079,0.116142,0.220130,0.478572,0.067239,0.894777,0.979164,0.387981,...,0.153317,0.099130,0.017873,0.342599,0.036244,0.063664,0.254674,0.192429,0.162536,0.324287
3,HPSI0813i-wots_3,380,0.170352,0.057083,0.207481,0.515186,0.075978,0.994246,1.021121,0.591241,...,0.315672,0.077485,0.025384,0.404380,0.036955,0.037978,0.378846,0.137812,0.125242,0.275611
4,HPSI0513i-dipe_1,475,0.206783,0.063727,0.195588,0.431934,0.088978,0.974636,0.988321,0.447228,...,0.283717,0.055024,0.026170,0.328487,0.034488,0.054598,0.245008,0.191966,0.126739,0.239621


In [46]:
#create merged index
phenotype_df['index'] = phenotype_df['donor_id']
cell_count_ds = phenotype_df['n_cells']
cell_count_ds.index = phenotype_df['index'].tolist()

In [47]:
#take donor and pool cols for samplemapping df
samplemapping_df = phenotype_df[['donor_id','index']]
samplemapping_df.head()
samplemapping_df.to_csv(sample_mapping_file, sep='\t', index=False, header=False)

In [48]:
#reorganise to just be expression data indexed by the merged index
cols_to_drop = list(set(phenotype_df.columns) & {'donor_id','celltype','n_cells','treatment','pool_id'})
phenotype_df = phenotype_df.drop(cols_to_drop, axis=1).set_index('index')
phenotype_df = phenotype_df.transpose()
phenotype_df.head()

index,HPSI0813i-mioj_1,HPSI1013i-zagm_1,HPSI0614i-lirf_5,HPSI0813i-wots_3,HPSI0513i-dipe_1,HPSI0914i-laey_6,HPSI1013i-garx_2,HPSI0914i-euts_3,HPSI0414i-uawq_2,HPSI1013i-jufd_3,...,HPSI1114i-kuul_1,HPSI0215i-uiao_2,HPSI0414i-xojn_3,HPSI1014i-vils_1,HPSI0115i-hecn_6,HPSI0314i-sojd_3,HPSI0513i-oarz_22,HPSI1113i-qorq_1,HPSI0614i-juzt_4,HPSI0813i-guss_1
ATG10,0.112087,0.244225,0.103079,0.170352,0.206783,0.179012,0.136026,0.178352,0.165347,0.059772,...,0.180358,0.152995,0.158102,0.148469,0.089599,0.129314,0.190318,0.203343,0.127675,0.099703
TGDS,0.101908,0.058878,0.116142,0.057083,0.063727,0.073507,0.059054,0.043088,0.075377,0.112744,...,0.088782,0.085682,0.060322,0.091500,0.060687,0.077597,0.043690,0.078528,0.093923,0.128516
MIB2,0.034625,0.364975,0.220130,0.207481,0.195588,0.220356,0.172584,0.259656,0.221951,0.044815,...,0.262320,0.212287,0.210651,0.178702,0.112425,0.165096,0.224049,0.326346,0.160031,0.035286
NRBP1,0.188004,0.291192,0.478572,0.515186,0.431934,0.551177,0.480069,0.533810,0.474848,0.195282,...,0.534081,0.538723,0.533000,0.479941,0.374597,0.418244,0.577636,0.502087,0.523326,0.170765
MFI2-AS1,0.063741,0.025435,0.067239,0.075978,0.088978,0.041596,0.072450,0.067155,0.078493,0.103441,...,0.095802,0.084611,0.089897,0.091380,0.090971,0.100960,0.117055,0.034351,0.102691,0.096047


In [49]:
gene_names = phenotype_df.index

In [50]:
#number of top expressed genes
selected_genes = list(phenotype_df.mean(axis=1).nlargest(n_genes).index)
phenotype_df = phenotype_df.loc[selected_genes, :]

In [51]:
phenotype_list = phenotype_df.index
mapping_df = mapping_df.query('hgnc_symbol in @phenotype_list')
mapping_df = mapping_df.drop_duplicates(subset=['hgnc_symbol'])
mapping_df = mapping_df.set_index('hgnc_symbol')

In [52]:
# limit only to hgnc symbols that map to ensembl gene IDs
phenotype_df = phenotype_df.loc[mapping_df.index,:]
phenotype_df.index = mapping_df['ensembl_gene_id']
phenotype_df.head()

index,HPSI0813i-mioj_1,HPSI1013i-zagm_1,HPSI0614i-lirf_5,HPSI0813i-wots_3,HPSI0513i-dipe_1,HPSI0914i-laey_6,HPSI1013i-garx_2,HPSI0914i-euts_3,HPSI0414i-uawq_2,HPSI1013i-jufd_3,...,HPSI1114i-kuul_1,HPSI0215i-uiao_2,HPSI0414i-xojn_3,HPSI1014i-vils_1,HPSI0115i-hecn_6,HPSI0314i-sojd_3,HPSI0513i-oarz_22,HPSI1113i-qorq_1,HPSI0614i-juzt_4,HPSI0813i-guss_1
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000261657,0.157152,0.293780,0.196034,0.144988,0.239364,0.198288,0.137375,0.181127,0.197648,0.191660,...,0.187689,0.226912,0.205653,0.188130,0.186459,0.215389,0.194516,0.230161,0.248207,0.140535
ENSG00000247400,0.090413,0.058130,0.056450,0.088196,0.090959,0.075289,0.057657,0.115350,0.073013,0.057998,...,0.086908,0.072532,0.094456,0.060830,0.084909,0.066078,0.111106,0.070322,0.075650,0.056412
ENSG00000228889,0.033071,0.051235,0.053569,0.016273,0.031071,0.031619,0.026846,0.031503,0.037756,0.058847,...,0.047876,0.040462,0.039612,0.044540,0.028943,0.037863,0.029506,0.044855,0.042405,0.043266
ENSG00000272143,0.089728,0.113941,0.103190,0.071098,0.086025,0.080396,0.075664,0.163828,0.094052,0.135909,...,0.077964,0.107662,0.095064,0.105976,0.112993,0.108685,0.061790,0.112781,0.107630,0.126240
ENSG00000261577,0.084997,0.196845,0.117513,0.140259,0.207238,0.174128,0.155906,0.197414,0.123122,0.060319,...,0.130926,0.164073,0.137374,0.083468,0.099059,0.083514,0.136721,0.226844,0.089507,0.061436


In [53]:
phenotype_df.to_csv(phenotype_file, sep='\t')

In [54]:
pc_mat = sklearn.decomposition.PCA(n_components=n_pcs).fit_transform(phenotype_df.values.transpose())
pc_df = pd.DataFrame(data=pc_mat, index=phenotype_df.columns, columns=['PC{}'.format(x) for x in range(1,n_pcs+1)])

pc_df.to_csv(covariates_file, sep='\t')

In [55]:
noise_scaling_vector = [1/float(x) for x in cell_count_ds.tolist()]
noise_matrix = np.diag(noise_scaling_vector)
noise_matrix_df = pd.DataFrame(data=noise_matrix, index=cell_count_ds.index, columns=cell_count_ds.index)

In [56]:
noise_matrix_df.to_csv(noise_matrix_file, sep='\t')
kinship_file = noise_matrix_file

In [57]:
number_of_permutations = '1000'
minor_allele_frequency = '0.05'
hwe = '0.000001'
call_rate = '1'
window_size = '250000'
block_size = '15000'


config_dict = {'af': annotation_file,
               'pf': phenotype_file,
               'cf': covariates_file,
               'kf': kinship_file,
               'smf': sample_mapping_file,
               'plink': genotypes_file,
               'maf': minor_allele_frequency,
               'hwe': hwe,
               'np': number_of_permutations,
               'cr': call_rate,
               'w': window_size,
               'bs': block_size,
               'chunk_file': chunk_file
}

# write config to a file
qtl_config_utils.write_config(config_dict, config_file)

NameError: name 'qtl_config_utils' is not defined